In [1]:
import sqlite3 as sq
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import polars as pl

In [42]:
con = sq.connect('../add-tournament/add-tournament/stats2.db')
qbr = sq.connect('strength.db')
pg = pl.read_database("SELECT tournament_id, set_id, team_id, round, game_id, player_id, pts FROM player_games", 
                      con).select(
    pl.col("tournament_id"),
    pl.col("set_id").cast(pl.Int16),
    pl.col("round"),
    pl.col("game_id").cast(pl.Int64),
    pl.col("team_id"),
    pl.col("player_id").cast(pl.Int32),
    pl.col("pts")
)
tg = pl.read_database("""
SELECT sets.year, tournaments.tournament_name, tournaments.date, team_games.* FROM team_games 
LEFT JOIN tournaments on team_games.tournament_id = tournaments.tournament_id
LEFT JOIN sets on team_games.set_id = sets.set_id
""", con, infer_schema_length = 1000)

In [43]:
pg

tournament_id,set_id,round,game_id,team_id,player_id,pts
i64,i16,str,i64,str,i32,i64
2108,352,"""Round 1""",113123,"""154-1""",24021,0
2108,352,"""Round 1""",113123,"""105-1""",14338,80
2108,352,"""Round 1""",113123,"""105-1""",21831,45
2108,352,"""Round 1""",113123,"""105-1""",4795,40
2108,352,"""Round 1""",113124,"""560-2""",23220,0
…,…,…,…,…,…,…
1999,334,"""All Rounds""",119084,"""252-6""",11950,20
1999,334,"""All Rounds""",119084,"""343-1""",1354,-5
1999,334,"""All Rounds""",119084,"""343-1""",1400,15


In [44]:
points_buffer = 20

initial_variability = 1
variability_dec = .99
variability_min = .05
var_daily_increase = .001

initial_qbr = 100

In [ ]:
tournaments = pl.read_database("SELECT * FROM tournaments", con, infer_schema_length=1000).with_columns(pl.col("date").str.to_date("%Y-%m-%d"))
sets = pl.read_database("SELECT * FROM sets", con).with_columns(pl.col("set_id").cast(pl.Int16))
player_list = pl.read_database("SELECT players.*, people.player FROM players left join people on players.person_id = people.person_id", con).with_columns(
    pl.col("player_id").cast(pl.Int32),
    pl.col("person_id").cast(pl.Int32)
)
nats_players = pg.join(
  tournaments[['tournament_id', 'date', 'tournament_name']], on = 'tournament_id'
  ).join(
    sets, on = 'set_id'
  ).filter(
      pl.col('difficulty') != 'easy'
  ).join(
      player_list, on = 'player_id'
  ).select(
      'date', 'tournament_id', 'tournament_name', 'round', 'game_id', 'team_id', 'player_id', 'person_id', 'player', 'pts'
  )
nats_players

date,tournament_id,tournament_name,round,game_id,team_id,player_id,person_id,player,pts
date,i64,str,str,i64,str,i32,i32,str,i64
2024-11-16,2108,"""2024 ACF Winter at Rochester""","""Round 1""",113123,"""154-1""",24021,21343,"""Not a player""",0
2024-11-16,2108,"""2024 ACF Winter at Rochester""","""Round 1""",113123,"""105-1""",14338,14185,"""Ryan Judge""",80
2024-11-16,2108,"""2024 ACF Winter at Rochester""","""Round 1""",113123,"""105-1""",21831,19957,"""Tony Boyle""",45
2024-11-16,2108,"""2024 ACF Winter at Rochester""","""Round 1""",113123,"""105-1""",4795,4717,"""David Neiman""",40
2024-11-16,2108,"""2024 ACF Winter at Rochester""","""Round 1""",113124,"""560-2""",23220,20543,"""Sammy Carstens""",0
…,…,…,…,…,…,…,…,…,…
2024-02-03,1999,"""2024 DI SCT at Tennessee""","""All Rounds""",119084,"""252-6""",11950,11813,"""Michael Zhou""",20
2024-02-03,1999,"""2024 DI SCT at Tennessee""","""All Rounds""",119084,"""343-1""",1354,1330,"""Andrew Ellis""",-5
2024-02-03,1999,"""2024 DI SCT at Tennessee""","""All Rounds""",119084,"""343-1""",1400,1376,"""Andrew Kelley""",15


In [109]:
nats_subset = nats_players.filter(game_id = 56552).with_columns(
    p = pl.lit("p")
  ).with_columns(
    pplayer_id = pl.concat_str([pl.col("p"), pl.col("player_id")])
  ).with_columns(
      pl.coalesce(["person_id", "pplayer_id"]).alias("comb_player_id")
  ).drop(
      ["p", "pplayer_id"]
  )
nats_subset

date,tournament_id,tournament_name,round,game_id,team_id,player_id,person_id,player,pts,comb_player_id
date,i64,str,str,i64,str,i32,i32,str,i64,str
2011-10-01,29,"""2011 MOO at Rock Bridge""","""Round 1""",56552,"""440-1""",8651,8534,"""John Cozort""",50,"""8534"""
2011-10-01,29,"""2011 MOO at Rock Bridge""","""Round 1""",56552,"""440-1""",1166,1147,"""Amit Jain""",10,"""1147"""
2011-10-01,29,"""2011 MOO at Rock Bridge""","""Round 1""",56552,"""440-1""",12180,12041,"""Molly Parsons""",-5,"""12041"""
2011-10-01,29,"""2011 MOO at Rock Bridge""","""Round 1""",56552,"""440-1""",7730,7625,"""James Carter""",10,"""7625"""
2011-10-01,29,"""2011 MOO at Rock Bridge""","""Round 1""",56552,"""183-1""",16051,15881,"""Tiesha Boeker""",10,"""15881"""
2011-10-01,29,"""2011 MOO at Rock Bridge""","""Round 1""",56552,"""183-1""",8328,8216,"""Jess Birdsell""",0,"""8216"""
2011-10-01,29,"""2011 MOO at Rock Bridge""","""Round 1""",56552,"""183-1""",718,708,"""Alex Generous""",10,"""708"""
2011-10-01,29,"""2011 MOO at Rock Bridge""","""Round 1""",56552,"""183-1""",14645,14491,"""Sameep Sheth""",0,"""14491"""


In [135]:
game_scores = pl.read_database("SELECT comb_player_id, player_id, person_id, last_date, old_qbr, variability, adjustment from player_ratings_py", 
                               qbr, infer_schema_length=None).lazy().with_columns(
    pl.col('player_id').cast(pl.Int64),
    pl.col('person_id').cast(pl.Int64),
    pl.col('comb_player_id').cast(pl.String),
    pl.col('last_date').cast(pl.Date, strict=False)
).filter(
    pl.col('player_id').is_in(nats_subset['player_id'].to_list()),
    pl.col('person_id').is_in(nats_subset['person_id'].to_list())
  ).group_by('comb_player_id').tail(1).with_columns(
      qbr = pl.max_horizontal(pl.col("old_qbr") + pl.col("adjustment"), 0)
    ).collect()
game_scores

comb_player_id,player_id,person_id,last_date,old_qbr,variability,adjustment,qbr
str,i64,i64,date,i64,f64,f64,f64
"""15881""",16051,15881,null,100,1.0,14.285714,114.285714
"""8216""",8328,8216,null,100,1.0,-16.0,84.0
"""7625""",7730,7625,null,100,1.0,-20.0,80.0
"""1147""",1166,1147,null,100,1.0,-15.0,85.0
"""12041""",12180,12041,null,100,1.0,-25.714286,74.285714
"""8534""",8651,8534,null,100,1.0,-5.714286,94.285714
"""14491""",14645,14491,null,100,1.0,-2.5,97.5
"""708""",718,708,null,100,1.0,4.0,104.0


In [ ]:
# # Update variability
# game['variability'] = (game['variability'] * variability_dec) + (var_daily_increase * game['days_since'])

# # Apply variability constraints
# game['variability'] = np.minimum(game['variability'], initial_variability)
# game['variability'] = np.maximum(game['variability'], variability_min)

# # Final coalesce for variability
# game['variability'] = game['variability'].fillna(initial_variability)

# # Step 2: Calculate exp_pts, game_score, and adjustment
# game['exp_pts'] = total_pts * (game['qbr'] + points_buffer) / (game['qbr'] + points_buffer).sum()
# game['game_score'] = game['pts'] - game['exp_pts']
# game['adjustment'] = game['game_score'] / (1 / game['variability'])

nats_subset

date,tournament_id,round,game_id,team_id,player_id,person_id,player,pts,comb_player_id,last_date,qbr,variability,days_since,exp_pts,game_score,adjustment
date,i64,str,i64,str,i32,i32,str,i64,str,date,f64,f64,i64,f64,f64,f64
2011-10-01,29,"""Round 1""",56552,"""440-1""",8651,8534,"""John Cozort""",50,"""8534""",null,94.285714,0.99,0,23.027105,26.972895,26.703166
2011-10-01,29,"""Round 1""",56552,"""440-1""",1166,1147,"""Amit Jain""",10,"""1147""",null,85.0,0.99,0,21.156153,-11.156153,-11.044591
2011-10-01,29,"""Round 1""",56552,"""440-1""",12180,12041,"""Molly Parsons""",-5,"""12041""",null,74.285714,0.99,0,18.997361,-23.997361,-23.757388
2011-10-01,29,"""Round 1""",56552,"""440-1""",7730,7625,"""James Carter""",10,"""7625""",null,80.0,0.99,0,20.148717,-10.148717,-10.04723
2011-10-01,29,"""Round 1""",56552,"""183-1""",16051,15881,"""Tiesha Boeker""",10,"""15881""",null,114.285714,0.99,0,27.056848,-17.056848,-16.88628
2011-10-01,29,"""Round 1""",56552,"""183-1""",8328,8216,"""Jess Birdsell""",0,"""8216""",null,84.0,0.99,0,20.954665,-20.954665,-20.745119
2011-10-01,29,"""Round 1""",56552,"""183-1""",718,708,"""Alex Generous""",10,"""708""",null,104.0,0.99,0,24.984409,-14.984409,-14.834565
2011-10-01,29,"""Round 1""",56552,"""183-1""",14645,14491,"""Sameep Sheth""",0,"""14491""",null,97.5,0.99,0,23.674742,-23.674742,-23.437995


In [176]:
cols = pl.read_database("SELECT * from player_ratings_py", qbr, infer_schema_length=10000).columns

game_scores_df = pl.DataFrame(
  schema={
    'date': pl.Date,
    'tournament_id': pl.Int16,
    'round': pl.String,
    'game_id': pl.Int64,
    'team_id': pl.String,
    'player_id': pl.Int32,
    'person_id': pl.Int32,
    'comb_player_id': pl.String,
    'player': pl.String,
    'old_qbr': pl.Float32,
    'variability': pl.Float32,
    'pts': pl.Int16,
    'exp_pts': pl.Float32,
    'game_score': pl.Float32,
    'adjustment': pl.Float32,
    'last_date': pl.Date,
    'days_since': pl.Int16
  }
)

def calculate_game_score(player_df, total_pts = 180):
  player_ids = player_df['player_id'].to_list()
  person_ids = player_df['person_id'].to_list()
  player_df = player_df.with_columns(
    p = pl.lit("p")
  ).with_columns(
    pplayer_id = pl.concat_str([pl.col("p"), pl.col("player_id")])
  ).with_columns(
      pl.coalesce(["person_id", "pplayer_id"]).alias("comb_player_id")
  ).drop(
      ["p", "pplayer_id"]
  )
  print(game_scores_df)
  game_scores = game_scores_df.lazy().filter(
    pl.col('player_id').is_in(player_ids),
    pl.col('person_id').is_in(person_ids)
  ).group_by(
    'comb_player_id'
    ).tail(1).with_columns(
      qbr = pl.max_horizontal(pl.col("old_qbr") + pl.col("adjustment"), 0)
    ).rename(
      {'date': 'last_date'}
      ).collect()

  game = player_df.lazy().drop(
    'tournament_name'
    ).join(
      game_scores.lazy().select('comb_player_id', 'last_date', 'qbr', 'variability'), 
      on = 'comb_player_id',
      how = 'left'
      ).with_columns(
          pl.col(["date", "last_date"]).cast(pl.Date)
      ).with_columns(
          days_since = (pl.col("date") - pl.col('last_date')).dt.total_days().fill_null(0),
          qbr = pl.col("qbr").fill_null(initial_qbr)
      ).with_columns(
          pl.min_horizontal(
              pl.max_horizontal(
                  pl.col("variability")*variability_dec + (var_daily_increase*pl.col("days_since")),
                  variability_min), 
              initial_variability).fill_null(initial_variability)
      ).with_columns(
          exp_pts = 180 * (pl.col("qbr") + points_buffer) / (pl.col("qbr") + points_buffer).sum()
      ).with_columns(
          game_score = pl.col("pts") - pl.col("exp_pts")
      ).with_columns(
          adjustment = pl.col("game_score") / (1 / pl.col("variability"))
      ).rename(
        {'qbr': 'old_qbr'}
      ).collect()
  game_scores_df.extend(game)
  game.write_database('player_ratings_py', qbr, if_table_exists = 'append')

In [177]:
calculate_game_score(nats_subset)

shape: (0, 17)
┌──────┬───────────────┬───────┬─────────┬───┬────────────┬────────────┬───────────┬────────────┐
│ date ┆ tournament_id ┆ round ┆ game_id ┆ … ┆ game_score ┆ adjustment ┆ last_date ┆ days_since │
│ ---  ┆ ---           ┆ ---   ┆ ---     ┆   ┆ ---        ┆ ---        ┆ ---       ┆ ---        │
│ date ┆ i16           ┆ str   ┆ i64     ┆   ┆ f32        ┆ f32        ┆ date      ┆ i16        │
╞══════╪═══════════════╪═══════╪═════════╪═══╪════════════╪════════════╪═══════════╪════════════╡
└──────┴───────────────┴───────┴─────────┴───┴────────────┴────────────┴───────────┴────────────┘


DuplicateError: column with name 'last_date' has more than one occurrence

In [158]:
cur = qbr.cursor()
cur.execute('DROP TABLE player_ratings_py')
# cur.execute('CREATE TABLE player_ratings (date, tournament_id, round, game_id, team_id, player_id, mu, sigma)')
cur.close()

In [14]:
game_ids = nats_players.drop_duplicates().sort_values(['date', 'game_id'])
ratings = pd.read_sql('SELECT * from player_ratings_py', qbr)
prev_games = [int(el) for el in ratings['game_id'].unique()]
game_ids = game_ids[~game_ids.game_id.isin(prev_games)]
game_ids

,date,tournament_id,tournament_name,round,game_id,team_id,player_id,person_id,player,pts
371379,2011-10-01,29,2011 MOO at Rock Bridge,Round 1,56552,183-1,718.0,708.0,Alex Generous,10
400673,2011-10-01,29,2011 MOO at Rock Bridge,Round 1,56552,440-1,8651.0,8534.0,John Cozort,50
400714,2011-10-01,29,2011 MOO at Rock Bridge,Round 1,56552,440-1,1166.0,1147.0,Amit Jain,10
400745,2011-10-01,29,2011 MOO at Rock Bridge,Round 1,56552,440-1,12180.0,12041.0,Molly Parsons,-5
400765,2011-10-01,29,2011 MOO at Rock Bridge,Round 1,56552,440-1,7730.0,7625.0,James Carter,10
...,...,...,...,...,...,...,...,...,...,...
9620,2024-11-30,2126,2024 ARCADIA at Waterloo,Round 12,118377,756-2,18238.0,17767.0,Caleb Ott,35
10167,2024-11-30,2126,2024 ARCADIA at Waterloo,Round 12,118377,687-1,16234.0,16062.0,Tony Chen,25
10738,2024-11-30,2126,2024 ARCADIA at Waterloo,Round 12,118377,687-1,6286.0,6193.0,Gareth Thorlakson,15
45320,2024-11-30,2126,2024 ARCADIA at Waterloo,Round 12,118377,687-1,2251.0,2366.0,Benjamin Chapman,50


In [15]:
ratings = pd.read_sql('SELECT * from player_ratings_py', qbr)
cur_tournament = ''
for i, row in game_ids.iterrows():
    if cur_tournament != row['tournament_name']:
        print(f"{row['date']}: {row['tournament_name']}")
        cur_tournament = row['tournament_name']
    calculate_game_score(nats_players[nats_players['game_id'] == row['game_id']])

2011-10-01: 2011 MOO at Rock Bridge
2011-10-08: MOO at Virginia
2011-10-08: 2011 MOO at Harvard
2011-10-08: MOO at Michigan
2011-10-09: MOO at Claremont Colleges
2011-10-15: MAGNI at South Carolina
2011-10-15: MAGNI at Valencia
2011-10-15: MOO at McMaster
2011-10-15: MOO at Minnesota
2011-10-22: MAGNI at George Mason
2011-10-22: MAGNI at Ohio State
2011-10-22: MAGNI at Carleton College
2011-10-24: MOO at Georgia Tech
2011-10-29: MAGNI at Brown
2011-10-29: MAGNI at UC San Diego
2011-10-29: MAGNI at Missouri S&T
2011-10-29: MAGNI at McMaster
2011-10-29: MAGNI at Brown
2011-11-19: 2011 Minnesota Open at Penn
2011-11-19: 2011 Minnesota Open at Minnesota


KeyboardInterrupt: 

In [16]:
len(prev_games)

24752